# Calculating yields and prediction of expression modulation targets

The following chapter deals with the calculation of product and biomass yields in relation to different carbon sources and the prediction of expression modulation targets.

If input is required by you, this is indicated as `None`.

In [ ]:
# file system and path operations
import os
import numpy as np
import matplotlib.pyplot as plt

# load wget and install it if necessary
try:
    import wget
except:
    print('Installing wget')
    !pip install wget
    import wget
# load cobra toolbox and install it if necessary
try:
    from cobra.io import read_sbml_model
    print('Cobra toolbox is already installed')
except:
    print('Installing cobra toolbox')
    !pip install cobra
    from cobra.io import read_sbml_model
print('Done')

Cobra toolbox is already installed
Done


## Loading a model

The next step is to download a genome-scale model. The `ecoli_core_model` serves as an example. This is available at http://bigg.ucsd.edu/models/e_coli_core. In the following cell the model is downloaded via the Python command `wget` and stored within the `Data` folder. You can now use this model for further work within the jupyter notebook.

In [ ]:
# check if the file 'e_coli_core.xml.gz' is in the `Data` directory
ModelFile = os.path.join('..', 'Data', 'e_coli_core.xml.gz')
if os.path.isfile(ModelFile):
    print('Loading existing file e_coli_core.xml.gz')
else:
    print('Download of file e_coli_core.xml.gz from BIGG')
    import wget
    # download the file from BIGG and save it in the `Data` directory
    wget.download('http://bigg.ucsd.edu/static/models/e_coli_core.xml.gz')
    # move the file to the `Data` directory
    os.rename('e_coli_core.xml.gz', ModelFile)
    print('Done')


## Product yield

To preserve the original model, a copy of the model is first created for the changes.

In [ ]:
model = model_original.copy()

Calculating the yield of a product is straigtforward. First one sets the producing reaction a new objective of the model.

In [ ]:
model.objective = 'EX_ac_e'

Then one determines the maximum production flux by solving the model.

In [ ]:
solution = model.optimize()
solution.fluxes['EX_ac_e']

Dividing the production flux by the uptake flux of the carbon source (in this case glucose) yields the molar yield (mol product / mol glucose).

In [ ]:
solution.fluxes['EX_ac_e'] / (-1. * solution.fluxes['EX_glc__D_e'])

## Exercise (15 min)
Convert the yield into a C-mol (product) / C-mol (carbon source) yield.

Calculate a mass yield instead of molar yield (g (product) / g (carbon source))

## Biomass Yield

The ratio of the amount of biomass produced to the amount of substrate consumed (g biomass/g substrate) is defined as the biomass yield, and typically is defined relative to the electron donor used.


In [ ]:
model = model_original.copy()

In [ ]:
solution = model.optimize()
g_dw_biomass_per_mmol_glucose = solution.objective_value / (-1. * solution.fluxes['EX_glc__D_e'])
g_dw_biomass_per_mmol_glucose

## Excercise (10 min)
The units of this yield are gDW (Biomass) / g (glucose). Convert this number into a mass yield.

In [ ]:
# Enter code here

## Growth vs. Product yield
## Exercise (30 min)
In the previous section we calculated theoretical maximum yields in the absence of competing objectives. But how does product yield vary with growth for example?

Determine molar, mass, and c-mol yields of acetate for the following growth rates (plot them using the code snippet below). How do the yields looks like for anaerobic conditions?

Follow the following steps:
 1. identify the model ID for the acetate exchange reaction
 2. set objective to acetate exchange reaction
 2. use a for loop to set the growth rate to the pre-defined value


In [ ]:
# find the reaction exchange id for the acetate here
model.reactions.query('None')

In [ ]:
# for loop to test production of acetate for a number of growth rates
# We test the following growth rates
growth_rates = [0.0, 0.10915242, 0.21830485, 0.32745727, 0.43660969, 0.54576212, 0.65491454, 0.76406697, 0.87321939, 0.98237181]
with model:
    model.objective = 'None'
    # an empty list for the product is initiated, which will store our results for each growth rate
    product_fluxes = list()
    for growth_rate in growth_rates:
        # the following line sets the upper and lower bound for the growth rate
        # replace the first None by the reaction ID, and the second two None by the growth rate in the loop
        model.reactions.get_by_id('None').bounds = None, None
        product_fluxes.append(model.slim_optimize())
# converting the product flux list into a numpy variable to simplify further calculations
product_fluxes = np.array(product_fluxes)

Fill in your own product fluxes here (needs to be a list).

In [ ]:
plt.plot(growth_rates, product_fluxes)
plt.xlabel('Growth [$h^{-1}$]')
plt.ylabel('Product flux [$mmol gDW^{-1} h^{-1}$]')
plt.show()

Fill in your own product yields here (needs to be a list).
Save the figure as png-type with a suitable name.

In [ ]:
yields = None # replace this with an equation for the yield
plt.plot(growth_rates, yields)
plt.xlabel('Growth [$h^{-1}$]')
plt.ylabel('Product yield [$mmol(acetate) / mmol(glucose)$]')

# Saving figure
plt.savefig('None.png')
plt.show()